In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



2024-04-01 11:35:47.933634: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 11:35:47.965740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 11:35:47.965766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 11:35:47.966634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 11:35:47.972021: I tensorflow/core/platform/cpu_feature_guar

In [5]:
import wandb
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers, models, optimizers

# Ensure wandb is installed: pip install wandb
# Initialize your W&B project before running the script

# Dataset loading and preparation
def load_and_prepare_dataset(batch_size):
    with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
        images = np.array(hdf.get('images'))
        boxes = np.array(hdf.get('boxes'))

    image_normalized = (images + 1e-9) / 9.26
    normalized_boxes = boxes / [1, 64, 64, 64, 64]

    images_np = image_normalized
    probabilities = np.array(normalized_boxes[:, :, :-4])
    probabilities = tf.expand_dims(probabilities, axis=1)
    boxes_np = np.array(normalized_boxes[:, :, 1:])
    boxes_np = tf.expand_dims(boxes_np, axis=1)

    dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)
    return dataset

# Model building
def build_model():
    input_shape = (64, 64, 1)
    num_classes = 280
    num_coordinates = 4

    x_input = layers.Input(shape=input_shape)
    #Layer 1
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 2
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 3
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 4
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    #Layer 5
    x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    
    x = layers.Flatten()(x)
    x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
    x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
    x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
    x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

    model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
    return model

# Optimizer selection
def select_optimizer(optimizer_name, learning_rate):
    if optimizer_name == 'adam':
        return optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer specified.")

# Training function with sweep
def train_with_sweep():
    # config_defaults = {
    #     'batch_size': 32,
    #     'epochs': 10,
    #     'optimizer': 'adam',
    #     'learning_rate': 0.001
    }
    # with wandb.init(config=config):
        config = wandb.config

        dataset = load_and_prepare_dataset(config.batch_size)
        model = build_model()

        optimizer = select_optimizer(config.optimizer, config.learning_rate)
        model.compile(optimizer=optimizer, 
                      loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
                      metrics={'x_prob_reshape': 'accuracy'})

        # Fit the model
        model.fit(dataset, epochs=config.epochs, callbacks=[wandb.keras.WandbCallback()])


SyntaxError: unmatched '}' (1480004830.py, line 85)

In [3]:

sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 1}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [8]:
sweep_config


{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['adam', 'sgd']},
  'fc_layer_size': {'values': [128, 256, 512]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'epochs': {'value': 1},
  'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},
  'batch_size': {'distribution': 'q_log_uniform_values',
   'q': 8,
   'min': 32,
   'max': 256}}}

In [4]:
sweep_id = wandb.sweep(sweep_config, project="Working Hyperparameter Tuning")
wandb.agent(sweep_id, train_with_sweep, count=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: y8wizmg0
Sweep URL: https://wandb.ai/alphanium/Working%20Hyperparameter%20Tuning/sweeps/y8wizmg0


wandb: Agent Starting Run: 7zdmv06h with config:
wandb: 	batch_size: 152
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.06493151529635642
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: derrickappiahosei (alphanium). Use `wandb login --relogin` to force relogin


2024-04-01 11:38:57.884303: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22135 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2024-04-01 11:39:00.006913: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-01 11:39:00.094740: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-01 11:39:01.291485: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or 

66/66 [==============================] - 27s 206ms/step - loss: 0.3675 - x_prob_reshape_loss: 0.2506 - x_boxes_reshape_loss: 0.1169 - x_prob_reshape_accuracy: 0.9000


...

66/66 [==============================] - 8s 83ms/step - loss: 0.8925 - x_prob_reshape_loss: 0.7778 - x_boxes_reshape_loss: 0.1146 - x_prob_reshape_accuracy: 0.7566


epoch,▁
loss,▁
x_boxes_reshape_loss,▁
x_prob_reshape_accuracy,▁
x_prob_reshape_loss,▁
epoch,0
loss,0.89247
x_boxes_reshape_loss,0.11464
x_prob_reshape_accuracy,0.75663
x_prob_reshape_loss,0.77783


wandb: Agent Starting Run: 0xk3mat6 with config:
wandb: 	batch_size: 88
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.005222724154517866
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


114/114 [==============================] - 18s 89ms/step - loss: 0.5866 - x_prob_reshape_loss: 0.4140 - x_boxes_reshape_loss: 0.1727 - x_prob_reshape_accuracy: 0.8256


epoch,▁
loss,▁
x_boxes_reshape_loss,▁
x_prob_reshape_accuracy,▁
x_prob_reshape_loss,▁
epoch,0
loss,0.58664
x_boxes_reshape_loss,0.17265
x_prob_reshape_accuracy,0.82563
x_prob_reshape_loss,0.41399
